Importing environment, state process function and DQN solver function

In [4]:
from puckworld import DQNSolver, puckworld

Puck World Implementation

In [5]:
from math import atan2, sqrt
import numpy as np
# function to convert game state to np array
    
def process_state(state):
    
    location_vec = np.array([state['player_x'],
                          state['player_y'],
                          state['player_velocity_x'],
                          state['player_velocity_y']])
    
    good_range = sqrt((state['player_x'] - state['good_creep_x'])**2 + (state['player_y'] - state['good_creep_y'])**2)
    
    bad_range = sqrt((state['player_x'] - state['bad_creep_x'])**2 + (state['player_y'] - state['bad_creep_y'])**2)
    
    
    range_vec = np.append(good_range, bad_range)
    
    good_bearing = atan2((state['player_x'] - state['good_creep_x']), (state['player_y'] - state['good_creep_y']))
    
    bad_bearing = atan2((state['player_x'] - state['bad_creep_x']), (state['player_y'] - state['bad_creep_y']))
    
    bearing_vec = np.append(good_bearing, bad_bearing)
    
    state_vec = np.concatenate([location_vec, range_vec, bearing_vec])
    
    return state_vec

In [6]:
puckworld(process_state = process_state)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Run: 1, exploration: 0.4036245882390106, score: -300.7723386994112
Run: 2, exploration: 0.14811319969530845, score: -196.02550039573543
Run: 3, exploration: 0.0543512971290831, score: -330.34543595639116
Run: 4, exploration: 0.019944633602480003, score: -98.89187284470245
Run: 5, exploration: 0.01, score: -165.71818592963245
Run: 6, exploration: 0.01, score: -127.0407554618796
Run: 7, exploration: 0.01, score: -323.80813959512307
Run: 8, exploration: 0.01, score: -144.14177911645675
Run: 9, exploration: 0.01, score: -65.22398147173953
Run: 10, exploration: 0.01, score: -70.87111786266905
Run: 11, exploration: 0.01, score: -92.17918317576914
Run: 12, exploration: 0.01, score: -93.33024923835183


SystemExit: 

C:\anaconda\envs\rl\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


NameError: name 'step' is not defined

Evaluate trained model

In [ ]:
game = WaterWorld(width=500, height=500, num_creeps=5)
p = PLE(game, display_screen=True, state_preprocessor=process_state)
#agent = myAgentHere(input_shape=p.getGameStateDims(), allowed_actions=p.getActionSet())

p.init()
nb_frames = 10000
reward = 0.0
rewards = []
for i in range(nb_frames):
    if p.game_over():
        p.reset_game()
    state = p.getGameState()
    #action = agent.pickAction(reward, state)
    action = rng.choice([119, 97, 100, 115])
    reward = p.act(action)